---
## Imports
---

In [1]:
import pandas as pd
import os
import json
from web3 import Web3
from pathlib import Path
from dotenv import load_dotenv
import streamlit as st

load_dotenv()

True

---
## Server Funcitonality
---

In [2]:
# Pull Log
# Evaluate traders
	# Trading period
	# Trading metrics
	# Send Eval Report (open source/show functions)
	
# Distribute Subscriptions


---
## Client Functionality
---

### Trader Functionality


In [3]:
from Libraries.web3_contract import load_contract
contract = load_contract(".\Contracts\ABI\Contract_abi.json")

In [9]:
# For interacting with the market (Opening/Closing Trades) utilize this method
# Easy to keep things consistent and trade out trading platforms
from Libraries.TradingPlatforms import trade_platforms, init_TradingPlatform
test_platform = init_TradingPlatform(trade_platforms['alpaca'],contract)
test_platform.__hello__()

'This is a Alapaca trading platform.'

In [5]:

# Receive Subscriptions

### Viewer Functionality


In [6]:
# Subscribe

---
## Client Interface
---


### Trader Interface


In [7]:
########## SideBar ##########
# Select Trade Type Dropdown
# Start Trading Button

########## Tabs ##########
# Enter Trade
# Close Trade (from radio or dropdown)

### Viewer Interface

In [8]:
# Enter Subscription
# View Subscriptions
	# Filter from logger and grader
